# Introduction

Read: http://aosabook.org/en/sqlalchemy.html

What you would basically do to get some data from Pythons DB-API:

    connection = dbapi.connect(user="user", pw="pw", host="host")
    cursor = connection.cursor()

    cursor.execute("select * from user_table where name=?", ("jack",))

    print "Columns in result:", [desc[0] for desc in cursor.description]

    for row in cursor.fetchall():
        print "Row:", row

    cursor.close()
    connection.close()

# How SQLAlchemy does it?

In [15]:
from sqlalchemy import create_engine

# Syntax: dialect[+dialect]://username:password@host/dbname
engine = create_engine("postgresql://alchemist@alchemist_db/alchemist")

result = engine.execute("SELECT version();")

print(result.fetchall())

[('PostgreSQL 9.5.0 on x86_64-pc-linux-gnu, compiled by gcc (Debian 4.9.2-10) 4.9.2, 64-bit',)]


*Note*: everything is handled behind the scenes!

### The Engine, Connection and ResultProxy

In [ ]:
# we can explicitly maintain a connection:
connection = engine.connect()  # Engine references a Dialect
print(type(engine))
print(type(connection))

In [26]:
result = connection.execute("SELECT current_date;")
print(type(result))

2016-02-06 21:02:07,073 INFO sqlalchemy.engine.base.Engine SELECT current_date;


INFO:sqlalchemy.engine.base.Engine:SELECT current_date;


2016-02-06 21:02:07,073 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


<class 'sqlalchemy.engine.result.ResultProxy'>


In [16]:
connection.close()  # but then we also must close it...

*Note*: slides

### Table and Column concepts

In [27]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table, Column, 
    Integer, String
)

metadata = MetaData()

# define a Table with some Columns
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String)
)

In [29]:
engine.echo = True  # I wan't to see the SQL statements
metadata.create_all(engine)

2016-02-06 21:02:52,260 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


INFO:sqlalchemy.engine.base.Engine:select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2016-02-06 21:02:52,261 INFO sqlalchemy.engine.base.Engine {'name': 'users'}


INFO:sqlalchemy.engine.base.Engine:{'name': 'users'}


## SQL expressions
SQLAlchemy renders the expressions as strings...

In [39]:
ins = users.insert()
print(str(ins))

INSERT INTO users (id, name) VALUES (:id, :name)


So, insert() renders the SQL INSERT with named bind parameters. When we provide values, only they get rendered:

In [51]:
ins = users.insert().values(name='Seven of Nine')
print(str(ins))

INSERT INTO users (name) VALUES (:name)


But where are is `Seven of Nine`?

In [42]:
print(str(ins.compile().params))

{'name': 'Seven of Nine'}


Let's add that to the DB:

In [52]:
connection = engine.connect()
result = connection.execute(ins)
print(type(result))
print(result.inserted_primary_key)

2016-02-06 21:15:08,862 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name) VALUES (%(name)s) RETURNING users.id


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name) VALUES (%(name)s) RETURNING users.id


2016-02-06 21:15:08,863 INFO sqlalchemy.engine.base.Engine {'name': 'Seven of Nine'}


INFO:sqlalchemy.engine.base.Engine:{'name': 'Seven of Nine'}


2016-02-06 21:15:08,863 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


<class 'sqlalchemy.engine.result.ResultProxy'>
[3]


INSERT can also be done like this:

In [53]:
ins = users.insert()
connection.execute(ins, id=17, name='Ellen Ripley')

2016-02-06 21:15:11,137 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


2016-02-06 21:15:11,137 INFO sqlalchemy.engine.base.Engine {'name': 'Ellen Ripley', 'id': 17}


INFO:sqlalchemy.engine.base.Engine:{'name': 'Ellen Ripley', 'id': 17}


2016-02-06 21:15:11,138 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


Or, we can insert many at once:

In [55]:
connection.execute(ins, [
    {'id': 18, 'name': 'Cameron Phillips'},
    {'id': 19, 'name': 'Sarah Connor'},
])

2016-02-06 21:15:49,909 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


2016-02-06 21:15:49,910 INFO sqlalchemy.engine.base.Engine ({'id': 18, 'name': 'Cameron Phillips'}, {'id': 19, 'name': 'Sarah Connor'})


INFO:sqlalchemy.engine.base.Engine:({'id': 18, 'name': 'Cameron Phillips'}, {'id': 19, 'name': 'Sarah Connor'})


2016-02-06 21:15:49,912 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id)=(18) already exists.
 [SQL: 'INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)'] [parameters: ({'id': 18, 'name': 'Cameron Phillips'}, {'id': 19, 'name': 'Sarah Connor'})]

### SELECT

In [59]:
from sqlalchemy import select
sel = select([users])
result = connection.execute(sel)

2016-02-06 21:17:06,928 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name 
FROM users


2016-02-06 21:17:06,929 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


We have the resultset (ResultProxy object), let''s see what is in it

In [61]:
for usr in result:
    print(usr)

We can access the fields in many ways:

In [63]:
result = connection.execute(sel)
usr = result.fetchone()
print('Name: {}'.format(usr['name']))
print('Name: {}'.format(usr[1]))
print('Name: {}'.format(usr[users.c.name]))

2016-02-06 21:18:19,585 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name 
FROM users


2016-02-06 21:18:19,585 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


Name: Seven of Nine
Name: Seven of Nine
Name: Seven of Nine


result.close() should be called now, as there are more results in there...

Pick the fields to select

In [64]:
sel = select([users.c.name])
result = connection.execute(sel)
for usr in result:
    print(usr)

2016-02-06 21:19:01,102 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name 
FROM users


2016-02-06 21:19:01,103 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


('Seven of Nine',)
('Ellen Ripley',)
('Cameron Phillips',)
('Sarah Connor',)


select() is really smart

In [67]:
# repeating for reference:
#
# users = Table('users', metadata,
#    Column('id', Integer, primary_key=True),
#    Column('name', String)
# ) 

from sqlalchemy import ForeignKey

address = Table('address', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
)

metadata.create_all(engine)


2016-02-06 21:21:16,306 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


INFO:sqlalchemy.engine.base.Engine:select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2016-02-06 21:21:16,307 INFO sqlalchemy.engine.base.Engine {'name': 'users'}


INFO:sqlalchemy.engine.base.Engine:{'name': 'users'}


2016-02-06 21:21:16,309 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE address (
	id SERIAL NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)




2016-02-06 21:21:16,311 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-02-06 21:21:16,359 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [68]:
connection.execute(users.delete()) # starting with fresh data

2016-02-06 21:22:57,907 INFO sqlalchemy.engine.base.Engine DELETE FROM users


INFO:sqlalchemy.engine.base.Engine:DELETE FROM users


2016-02-06 21:22:57,908 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-02-06 21:22:57,909 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [70]:
connection.execute(users.insert(), [
    {'id': 1, 'name': 'Seven of Nine'},
    {'id': 2, 'name': 'Cameron Phillips'},
    {'id': 3, 'name': 'Sarah Connor'},
])

connection.execute(address.insert(), [
    {'user_id': 1, 'email_address': 'seven_of_nine@voyager.dq'},
    {'user_id': 1, 'email_address': 'seven_of_nine@unimatrix.01'},
    {'user_id': 2, 'email_address': 'cameron@sky.net'},
    {'user_id': 3, 'email_address': 'sarah@sky.net'},
])


2016-02-06 21:23:42,674 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)


2016-02-06 21:23:42,674 INFO sqlalchemy.engine.base.Engine ({'name': 'Seven of Nine', 'id': 1}, {'name': 'Cameron Phillips', 'id': 2}, {'name': 'Sarah Connor', 'id': 3})


INFO:sqlalchemy.engine.base.Engine:({'name': 'Seven of Nine', 'id': 1}, {'name': 'Cameron Phillips', 'id': 2}, {'name': 'Sarah Connor', 'id': 3})


2016-02-06 21:23:42,675 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id)=(1) already exists.
 [SQL: 'INSERT INTO users (id, name) VALUES (%(id)s, %(name)s)'] [parameters: ({'name': 'Seven of Nine', 'id': 1}, {'name': 'Cameron Phillips', 'id': 2}, {'name': 'Sarah Connor', 'id': 3})]

In [72]:
for row in connection.execute(select([users, address])):
    print(row)

2016-02-06 21:23:56,307 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, address.id, address.user_id, address.email_address 
FROM users, address


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, address.id, address.user_id, address.email_address 
FROM users, address


2016-02-06 21:23:56,308 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


(1, 'Seven of Nine', 1, 1, 'seven_of_nine@voyager.dq')
(1, 'Seven of Nine', 2, 1, 'seven_of_nine@unimatrix.01')
(1, 'Seven of Nine', 3, 2, 'cameron@sky.net')
(1, 'Seven of Nine', 4, 3, 'sarah@sky.net')
(2, 'Cameron Phillips', 1, 1, 'seven_of_nine@voyager.dq')
(2, 'Cameron Phillips', 2, 1, 'seven_of_nine@unimatrix.01')
(2, 'Cameron Phillips', 3, 2, 'cameron@sky.net')
(2, 'Cameron Phillips', 4, 3, 'sarah@sky.net')
(3, 'Sarah Connor', 1, 1, 'seven_of_nine@voyager.dq')
(3, 'Sarah Connor', 2, 1, 'seven_of_nine@unimatrix.01')
(3, 'Sarah Connor', 3, 2, 'cameron@sky.net')
(3, 'Sarah Connor', 4, 3, 'sarah@sky.net')


Whoa, a Cartesian product :), we need a WHERE in there

In [73]:
sel = select([users, address]).where(users.c.id == address.c.user_id)
for row in connection.execute(sel):
    print(row)

2016-02-06 21:24:23,086 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, address.id, address.user_id, address.email_address 
FROM users, address 
WHERE users.id = address.user_id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, address.id, address.user_id, address.email_address 
FROM users, address 
WHERE users.id = address.user_id


2016-02-06 21:24:23,087 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


(1, 'Seven of Nine', 1, 1, 'seven_of_nine@voyager.dq')
(1, 'Seven of Nine', 2, 1, 'seven_of_nine@unimatrix.01')
(2, 'Cameron Phillips', 3, 2, 'cameron@sky.net')
(3, 'Sarah Connor', 4, 3, 'sarah@sky.net')


Did you get the `where()` thingie?!  What does `==` do?

Check more examples: http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#operators